# やること

## パラメータは？

パラメータとして与えるのは以下

①　$\rho_{\chi_{k}}$　:　 $\chi_{k}$に対するbeliefのprior precision。　candidateごとに用意する。

②　$\mu_{\chi_{k}}$　：　$\chi_{k}$に対するbeliefのmean。candidateごとに用意する。

③　$\rho_{\eta}$　：　シグナルのprecision。一つで良い。

④　$\chi_{k}$　：　candidate kのtrue quality。candidateごとに用意する。

## belief updating

③と④をgivenとして$s_{1,k}\sim N(\chi_{k}, \frac{1}{\rho_{\eta}})$をdrawする。

1期のシグナルをもとに、①②③を用いてposteriorを次のように構成する。

$B_{1, k} \sim N\left(\frac{\rho_{\chi_{k}} \mu_{\chi_{k}} + \rho_{\eta}s_{1, k}}{\rho_{\chi_{k}} + t \rho_{\eta}}, \frac{1}{\rho_{\chi_{k}} + t \rho_{\eta}}\right)$

このposterior meanを$E\left[\chi_{k}|\Omega_{1}\right]$として記録する。

以降t期においても同様に

③と④をgivenとして$s_{t,k}\sim N(\chi_{k}, \frac{1}{\rho_{\eta}})$をdrawする。

t期のシグナルをもとに、①②③を用いてposteriorを次のように構成する。

$B_{1, k} \sim N\left(\frac{\rho_{\chi_{k}} \mu_{\chi_{k}} + \rho_{\eta} \sum_{\tau = 1}^{t} s_{\tau, k}}{\rho_{\chi_{k}} + t \rho_{\eta}}, \frac{1}{\rho_{\chi_{k}} + t \rho_{\eta}}\right)$

そしてそのposterior meanを$E\left[\chi_{k}|\Omega_{t}\right]$として記録する。

## 詳細

パラメータセット一つにつきT回シグナルを乱数発生する。使うパラメータはいつも一緒なので出てくる数字に大差はない。

違うのはその出てくる順番。T!順あり得るので一つのパラメータセットにつきT!回1期からT期までの$E\left[\chi_{k}|\Omega_{t}\right]$を記録する。

が、階乗はあまりに大きすぎるので、100より大きい場合は100を採用する。

さらにパラメータは100セット用意。

以下では、candidate4人について$E\left[\chi_{k}|\Omega_{t}\right]$のpathを100×T!通り作成するためのclassを作成する。

使用するクラスは以下で定義されている。

class ExpectedChi:
    
    def __init__(self, lam = 1.0, T = 5, sim = 10, random_state = 1):
        self.lam = lam # parameter of exponential distribution which rho's follow
        self.T = T # time
        self.sim = sim # simulation number
        self.random_state = random_state
    
    def setparam(self):
        # params[0, :]　：　rho_eta
        # params[(1,2,3,4), :] : rho_chi for each candidate
        # params[(5,6,7,8), :] : mu_chi for each candidate
        # params[(9,10,11,12), :] : chi for each candidate
        r = np.random.RandomState(self.random_state)
        rho = r.exponential(self.lam, (5, self.sim))
        mu_chi = r.normal(0.0,1.0,(8, self.sim))
        params = np.vstack((rho, mu_chi))
        return params
        
    def signals(self):
        # make the 4 dimentional array : all_signals
        # each element in all_signals is 3 dimentional array whose shape is (T! * 4 * T )
        params = self.setparam()
        all_signals = np.empty([self.sim, min(100, factorial(self.T).astype(int)), 4, self.T])
        g = np.random.RandomState(self.random_state)
        for i in range(self.sim):
            for k in range(min(100, factorial(self.T).astype(int))):
                for j in range(4):
                    all_signals[i, k, j, :] = g.normal(params[j + 9, i], 1/params[0, i],  (1, self.T))
        return all_signals
    
    def cum_signals(self):
        # calculating cumulative signals for expected chi_k given omega
        sig = self.signals()
        for i in range(self.sim):
            for j in range(self.T):
                if j != 0:
                    sig[i, :, :, :][:, :, j] = sig[i, :, :, :][:, :, j] + sig[i, :, :, :][:, :, j -1]
        return sig
            
    def expected_chi(self):
        # calculating expected chi_k given omega
        cum = self.cum_signals()
        params = self.setparam()
        transform = np.empty((self.sim, min(100, factorial(self.T).astype(int)), 4, self.T))
        
        # first multiply cum by rho_eta 
        for i in range(self.sim):
            transform[i, :, :, :] = cum[i, :, :, :] * params[0, i]
            
        # second add rho_chi + mu_chi to transform
        for i in range(self.sim):
            for j in range(4):
                transform[i, :, j, :] = transform[i, :, j, :] + (params[j + 1, i] * params[j + 5, i])
        
        # third divide by rho_chi + t * rho_eta
        for i in range(self.sim):
            for j in range(4):
                for t in range(self.T):
                    transform[i, :, j, t] = transform[i, :, j, t] / (params[j + 1, i] + t * params[0, i])
        
        return transform
    
    def means(self, each_sim = True):
        # calculate mean
        if each_sim == True:
            # calculate the means of expected chi for each parameter set and t
            # return matrix
            return self.expected_chi().mean(axis = (1,2))
        
        else:
            # calcualte the over all mean
            return self.expected_chi().mean(axis = (0,1,2))
        
    def variances(self, each_sim = True):
        # calculate variance
        if each_sim == True:
            # calculate the variances of expected chi for each parameter set and t
            # return list
            return self.expected_chi().var(axis = (1,2))
        
        else:
            # calcualte the over all variances
            return self.expected_chi().var(axis = (0,1,2))

In [2]:
import numpy as np
from scipy.special import factorial
from ExpectedChi import ExpectedChi as EC
test = EC()

In [5]:
test.means()

array([[-0.95451902, -0.19665242, -0.06700772,  0.00919043,  0.01810542],
       [ 0.13738635,  0.36760502,  0.36495195,  0.37563575,  0.3661167 ],
       [-0.13912021, -0.30731005, -0.2098157 , -0.06743069,  0.03291647],
       [-0.10813287, -0.16120758, -0.1870669 , -0.207069  , -0.24767411],
       [ 0.20200643,  0.40211636,  0.36988999,  0.45764656,  0.38410938],
       [ 1.08042733,  1.04125474,  0.93038106,  0.88075584,  0.90492917],
       [ 0.61427264,  0.65103072,  0.64995042,  0.54578841,  0.4935955 ],
       [ 3.9227891 ,  0.93822636,  0.836572  ,  0.80312989,  0.81263714],
       [-2.35641226, -0.45330074, -0.35849907, -0.29776484, -0.24381213],
       [ 0.24563887,  0.41996209,  0.36431147,  0.38138495,  0.38177583]])

In [6]:
test.means(each_sim = False)

array([ 0.26443364,  0.27017245,  0.26936675,  0.28812673,  0.29026994])

In [7]:
test.variances()

array([[  2.70573909e+01,   1.71569905e+00,   1.04707228e+00,
          7.50241566e-01,   5.75372365e-01],
       [  2.51843678e+00,   1.01955946e+00,   7.64925646e-01,
          6.29940682e-01,   5.65279596e-01],
       [  9.95466515e+00,   1.64390898e+01,   2.43685028e+01,
          3.47867903e+01,   4.35889841e+01],
       [  9.12274113e-01,   9.33901670e-01,   8.61612411e-01,
          8.70316849e-01,   8.51200308e-01],
       [  3.84417408e+02,   2.29267606e+01,   1.15304060e+01,
          7.62365280e+00,   5.63466132e+00],
       [  3.25350271e+00,   3.84807498e+00,   3.86107561e+00,
          3.87465185e+00,   3.77069067e+00],
       [  3.14950507e+01,   7.38636046e+00,   4.60463474e+00,
          3.82735111e+00,   3.15152994e+00],
       [  2.24226373e+02,   5.05099907e+00,   2.36863369e+00,
          1.64310560e+00,   1.44482906e+00],
       [  7.50248567e+02,   8.10202397e+00,   4.05476539e+00,
          2.72201243e+00,   2.12856730e+00],
       [  2.22691658e+01,   2.0036404

In [9]:
test.variances(each_sim = False)

array([ 147.92013641,    7.19222793,    5.65475504,    5.91395317,
          6.38487564])

In [18]:
# 100が採用されている
test.expected_chi().shape

(10, 100, 4, 5)

## 実際に使用するシミュレーションを出力

In [3]:
chiom = EC(T = 14, sim = 100, )

In [21]:
chiom.means(each_sim = False)

array([ 1.16051465,  0.05088978,  0.0494573 ,  0.04283046,  0.05479632,
        0.06153513,  0.06707816,  0.0665279 ,  0.05256553,  0.06468128,
        0.05511648,  0.06044397,  0.06487856,  0.0593507 ])

In [22]:
chiom.variances(each_sim = False)

array([ 12245.64415593,     21.67245879,     15.4566816 ,     17.16466401,
           17.3042649 ,     19.15097192,     21.33564027,     23.20854254,
           24.26826882,     26.91330901,     29.53060822,     31.67632901,
           35.68377826,     36.80298698])

In [23]:
chiom.expected_chi().shape

(100, 100, 4, 14)

In [30]:
# 1次元配列にする
# 以下からわかるようにravelは、パラメータセット1組目の、並び替え順1回目の、候補者1の1〜14期から順に出力する。
print(np.ravel(chiom.expected_chi())[0:30])

[ 2.89129588  0.10339334 -0.58758511 -1.17881917 -0.76122864 -1.51305828
 -0.92781754 -1.09451194 -0.99379197 -1.02199736 -0.74943342 -1.07966752
 -1.10511463 -1.13517343  1.49313796  0.8845636   0.66442966  0.34298538
  0.26718983  0.30063936  0.05838547  0.18605492  0.26165906  0.27607344
  0.33608531  0.21150141  0.16030268  0.0334484   1.65060108  1.5363441 ]


In [32]:
# 出力
import pandas as pd
simu_chi = pd.Series(np.ravel(chiom.expected_chi()))
simu_chi.to_csv('chidata.csv')

In [5]:
# パラメータの初期値を出力する
import pandas as pd
learning_params = pd.Series(np.ravel(chiom.setparam()))
learning_params.to_csv('learnin_params.csv')